I could reach 0.77 accuracy; further manipulation with hyperparameters does not lead to anything in particular. <br />
For some reason kaggle cannot download the bert model from hugging face, so if you want to test it, you better do it in google colaboratory or on your machine

## Imports

In [1]:
#!g1.1
import pandas as pd
import numpy as np
import torch, transformers, time, datetime, random

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No GPU available, using the CPU instead.


## Prepare data

In [3]:
# data = pd.read_csv('../Data/Youtube_thumbnails/metadata.csv')
data = pd.read_csv('/Users/juniverse/Desktop/pointcloud/VectorUniverse/Data/Youtube_thumbnails/metadata.csv')

In [10]:
data.tail(2)

,Id,Channel,Category,Title
2513,DUg2SWWK18I,Web Dev Simplified,Tech,MVC Explained in 4 Minutes
2514,EumXak7TyQ0,Web Dev Simplified,Tech,Why I Don't Use Else When Programming


In [4]:
cat_map = {}
cat_list = data["Category"].unique().tolist()
for i in range(len(cat_list)):
    cat_map[cat_list[i]] = i

print(cat_map)

{'Science': 0, 'News': 1, 'Food': 2, 'Blog': 3, 'Tech': 4, 'Informative': 5, 'Comedy,Entertainment': 6, 'Entertainment': 7, 'Automobile': 8, 'Tech,Informative': 9, 'Automobile,Comedy': 10, 'VideoGames': 11, 'Food,Entertainment': 12, 'Blog,Comedy': 13, 'Comedy,Informative': 14, 'Tech,Comedy': 15, 'Comedy': 16, 'Blog,Science': 17, 'Blog,Entertainment': 18, 'Entertainment,Comedy': 19, 'Tech,News': 20, 'Entertainment,Blog': 21}


In [12]:
data["cat_code"] = data["Category"].apply(lambda x: cat_map[x])

data.cat_code

0       0
1       0
2       0
3       0
4       0
       ..
2510    4
2511    4
2512    4
2513    4
2514    4
Name: cat_code, Length: 2515, dtype: int64

In [13]:
data.Title

0                 The hardest problem on the hardest test
1       But what is a Fourier series? From heat flow t...
2                     But how does bitcoin actually work?
3       But what is a neural network? | Chapter 1, Dee...
4         The most unexpected answer to a counting puzzle
                              ...                        
2510                           JavaScript Form Validation
2511                 Learn DOM Manipulation In 18 Minutes
2512    Learn useState In 15 Minutes - React Hooks Exp...
2513                           MVC Explained in 4 Minutes
2514                Why I Don't Use Else When Programming
Name: Title, Length: 2515, dtype: object

In [14]:
data.Category.value_counts()

Category
Science                 323
Food                    281
VideoGames              265
Informative             218
News                    218
Automobile              197
Tech                    193
Blog                    160
Comedy,Entertainment     88
Entertainment,Comedy     88
Automobile,Comedy        85
Blog,Comedy              56
Comedy,Informative       56
Entertainment            55
Comedy                   30
Entertainment,Blog       30
Tech,Comedy              29
Blog,Entertainment       29
Tech,Informative         29
Tech,News                29
Blog,Science             28
Food,Entertainment       28
Name: count, dtype: int64

In [15]:
data[data["Category"] == "VideoGames"]

,Id,Channel,Category,Title,cat_code
718,9vrqKAiRDcM,EpicNameBro,VideoGames,Dark Souls - Armor Guide: Special Armors,11
719,JyfeBHAfpZQ,EpicNameBro,VideoGames,Bloodborne with ENB - 001 - Central Yharnam,11
720,KUqg8XtAHnc,EpicNameBro,VideoGames,Let's Play Dark Souls 1: From the Dark part 1,11
721,QrdwpKzNge0,EpicNameBro,VideoGames,"Bloodborne with ENB - 003 - Iosefka Dialogue, ...",11
722,A4g2pdjxnY8,EpicNameBro,VideoGames,Dark Souls - Armor Guide: Light Sets 1/2,11
...,...,...,...,...,...
2428,dDagv6SA8nw,videogamedunkey,VideoGames,Knack 2 (dunkview),11
2429,CCkPhYiKbJU,videogamedunkey,VideoGames,Old Dunkey and Cuphead,11
2430,6rjYXEYpuDo,videogamedunkey,VideoGames,Assassin's Creed : EA Origins,11
2431,pMPomCWUraQ,videogamedunkey,VideoGames,Metal Gear Dies,11


In [27]:
X = data.Title.apply(lambda x: ''.join([i for i in x if not i.isdigit()])).tolist() # no numbers
y = data.cat_code.tolist()

In [30]:
testxxx = list(data.Title)
for i in range(10): 
    if X[i]==testxxx[i]:
        print('yes')
    else:
        print('no')
        # print(X[i])
        # print()
        # print(list(testxxx)[i])


yes
no
yes
no
yes
yes
yes
yes
yes
no


In [31]:
#!g1.1
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Loading BERT tokenizer...


In [53]:
#!g1.1
encoded_dict = tokenizer(
                    X,
                    add_special_tokens=True,
                    max_length=64,
                    truncation=True,
                    padding=True,
                    return_tensors='pt'
               )

input_ids = [encoded_dict['input_ids']]
attention_masks = [encoded_dict['attention_mask']]

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y)

In [57]:
encoded_dict

{'input_ids': tensor([[  101,  1996, 18263,  ...,     0,     0,     0],
        [  101,  2021,  2054,  ...,     0,     0,     0],
        [  101,  2021,  2129,  ...,     0,     0,     0],
        ...,
        [  101,  4553,  3594,  ...,     0,     0,     0],
        [  101, 19842,  2278,  ...,     0,     0,     0],
        [  101,  2339,  1045,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [55]:
input_ids.size(), attention_masks.size(), labels.size()

(torch.Size([2515, 29]), torch.Size([2515, 29]), torch.Size([2515]))

In [58]:
#!g1.1
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset)) # 0.67
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

2,012 training samples
  503 validation samples


In [69]:
for x,y,c in dataset:
    print(x.shape,y.shape,c)
    break

torch.Size([29]) torch.Size([29]) tensor(0)


In [71]:
#!g1.1
batch_size = 16 # 4 - 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

## Prepare model

In [72]:
#!g1.1
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(cat_list),
    output_attentions = False,
    output_hidden_states = False
)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [73]:
#!g1.1
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5
                  eps = 1e-8 # default is 1e-8.
                )

/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [74]:
#!g1.1
epochs = 1 # 2 - 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

## Train

In [75]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [79]:
from tqdm import tqdm
#!g1.1

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

total_t0 = time.time()

for epoch_i in tqdm(range(0, epochs)):
    print(f'\n======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    t0 = time.time()
    
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        logits = output.logits

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    
    training_time = format_time(time.time() - t0)

    print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    print("\nRunning Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            output = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = output.loss
            logits = output.logits
            
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  0%|          | 0/1 [00:00<?, ?it/s]


======== Epoch 1 / 1 ========
Training...
  Batch    40  of    126.    Elapsed: 0:02:00.
  Batch    80  of    126.    Elapsed: 0:04:02.
  Batch   120  of    126.    Elapsed: 0:06:01.

  Average training loss: 2.52
  Training epoch took: 0:06:19

Running Validation...


100%|██████████| 1/1 [06:40<00:00, 400.94s/it]

  Accuracy: 0.36
  Validation Loss: 2.44
  Validation took: 0:00:22

Training complete!
Total training took 0:06:41 (h:mm:ss)


## Some tests

In [ ]:
#!g1.1
encoded_dict = tokenizer(
                    ["Economic war between China and USA continues", "ELDEN RING world record; game passed in 2 minutes", "calculus in a nutshell"],
                    add_special_tokens=True,
                    max_length=128,
                    truncation=True,
                    padding=True,
                    return_tensors='pt'
               )

input_ids = torch.cat([encoded_dict['input_ids']], dim=0)
attention_masks = torch.cat([encoded_dict['attention_mask']], dim=0)

prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
#!g1.1
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()

predictions = []

for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
  
    b_input_ids, b_input_mask = batch
  
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()

    predictions = predictions + list(np.argmax(logits, axis = 1)) 

print('    DONE.')

In [ ]:
predictions

In [ ]:
[cat_list[i] for i in predictions]